# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fab58559250>
├── 'a' --> tensor([[ 0.2163, -0.1810, -0.8099],
│                   [-1.6821,  1.0561,  1.8823]])
└── 'x' --> <FastTreeValue 0x7fab58559220>
    └── 'c' --> tensor([[-1.7257, -0.5643, -0.3352, -1.6255],
                        [-0.1530,  0.7911,  0.3412,  0.1411],
                        [ 0.2110,  0.8384,  0.4537,  1.5757]])

In [4]:
t.a

tensor([[ 0.2163, -0.1810, -0.8099],
        [-1.6821,  1.0561,  1.8823]])

In [5]:
%timeit t.a

63.9 ns ± 0.0228 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fab58559250>
├── 'a' --> tensor([[-1.0750,  0.1162, -0.5139],
│                   [ 0.3919,  1.0738, -1.8674]])
└── 'x' --> <FastTreeValue 0x7fab58559220>
    └── 'c' --> tensor([[-1.7257, -0.5643, -0.3352, -1.6255],
                        [-0.1530,  0.7911,  0.3412,  0.1411],
                        [ 0.2110,  0.8384,  0.4537,  1.5757]])

In [7]:
%timeit t.a = new_value

63.5 ns ± 0.0401 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.2163, -0.1810, -0.8099],
               [-1.6821,  1.0561,  1.8823]]),
    x: Batch(
           c: tensor([[-1.7257, -0.5643, -0.3352, -1.6255],
                      [-0.1530,  0.7911,  0.3412,  0.1411],
                      [ 0.2110,  0.8384,  0.4537,  1.5757]]),
       ),
)

In [10]:
b.a

tensor([[ 0.2163, -0.1810, -0.8099],
        [-1.6821,  1.0561,  1.8823]])

In [11]:
%timeit b.a

59 ns ± 0.0374 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.2250, -0.3141, -0.3334],
               [-0.0339, -1.0171, -0.6386]]),
    x: Batch(
           c: tensor([[-1.7257, -0.5643, -0.3352, -1.6255],
                      [-0.1530,  0.7911,  0.3412,  0.1411],
                      [ 0.2110,  0.8384,  0.4537,  1.5757]]),
       ),
)

In [13]:
%timeit b.a = new_value

479 ns ± 0.104 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

853 ns ± 0.302 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.4 µs ± 37.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

138 µs ± 161 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

140 µs ± 325 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7faaad23fd00>
├── 'a' --> tensor([[[ 0.2163, -0.1810, -0.8099],
│                    [-1.6821,  1.0561,  1.8823]],
│           
│                   [[ 0.2163, -0.1810, -0.8099],
│                    [-1.6821,  1.0561,  1.8823]],
│           
│                   [[ 0.2163, -0.1810, -0.8099],
│                    [-1.6821,  1.0561,  1.8823]],
│           
│                   [[ 0.2163, -0.1810, -0.8099],
│                    [-1.6821,  1.0561,  1.8823]],
│           
│                   [[ 0.2163, -0.1810, -0.8099],
│                    [-1.6821,  1.0561,  1.8823]],
│           
│                   [[ 0.2163, -0.1810, -0.8099],
│                    [-1.6821,  1.0561,  1.8823]],
│           
│                   [[ 0.2163, -0.1810, -0.8099],
│                    [-1.6821,  1.0561,  1.8823]],
│           
│                   [[ 0.2163, -0.1810, -0.8099],
│                    [-1.6821,  1.0561,  1.8823]]])
└── 'x' --> <FastTreeValue 0x7faaad23f7f0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.5 µs ± 38.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7faaad23f760>
├── 'a' --> tensor([[ 0.2163, -0.1810, -0.8099],
│                   [-1.6821,  1.0561,  1.8823],
│                   [ 0.2163, -0.1810, -0.8099],
│                   [-1.6821,  1.0561,  1.8823],
│                   [ 0.2163, -0.1810, -0.8099],
│                   [-1.6821,  1.0561,  1.8823],
│                   [ 0.2163, -0.1810, -0.8099],
│                   [-1.6821,  1.0561,  1.8823],
│                   [ 0.2163, -0.1810, -0.8099],
│                   [-1.6821,  1.0561,  1.8823],
│                   [ 0.2163, -0.1810, -0.8099],
│                   [-1.6821,  1.0561,  1.8823],
│                   [ 0.2163, -0.1810, -0.8099],
│                   [-1.6821,  1.0561,  1.8823],
│                   [ 0.2163, -0.1810, -0.8099],
│                   [-1.6821,  1.0561,  1.8823]])
└── 'x' --> <FastTreeValue 0x7faaad23f5e0>
    └── 'c' --> tensor([[-1.7257, -0.5643, -0.3352, -1.6255],
                        [-0.1530,  0.7911,  0.3412,  0.1411],
                 

In [23]:
%timeit t_cat(trees)

29.7 µs ± 78.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

58.4 µs ± 37.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.2163, -0.1810, -0.8099],
                [-1.6821,  1.0561,  1.8823]],
       
               [[ 0.2163, -0.1810, -0.8099],
                [-1.6821,  1.0561,  1.8823]],
       
               [[ 0.2163, -0.1810, -0.8099],
                [-1.6821,  1.0561,  1.8823]],
       
               [[ 0.2163, -0.1810, -0.8099],
                [-1.6821,  1.0561,  1.8823]],
       
               [[ 0.2163, -0.1810, -0.8099],
                [-1.6821,  1.0561,  1.8823]],
       
               [[ 0.2163, -0.1810, -0.8099],
                [-1.6821,  1.0561,  1.8823]],
       
               [[ 0.2163, -0.1810, -0.8099],
                [-1.6821,  1.0561,  1.8823]],
       
               [[ 0.2163, -0.1810, -0.8099],
                [-1.6821,  1.0561,  1.8823]]]),
    x: Batch(
           c: tensor([[[-1.7257, -0.5643, -0.3352, -1.6255],
                       [-0.1530,  0.7911,  0.3412,  0.1411],
                       [ 0.2110,  0.8384,  0.4537,  1.5757]],
         

In [26]:
%timeit Batch.stack(batches)

78.4 µs ± 157 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.2163, -0.1810, -0.8099],
               [-1.6821,  1.0561,  1.8823],
               [ 0.2163, -0.1810, -0.8099],
               [-1.6821,  1.0561,  1.8823],
               [ 0.2163, -0.1810, -0.8099],
               [-1.6821,  1.0561,  1.8823],
               [ 0.2163, -0.1810, -0.8099],
               [-1.6821,  1.0561,  1.8823],
               [ 0.2163, -0.1810, -0.8099],
               [-1.6821,  1.0561,  1.8823],
               [ 0.2163, -0.1810, -0.8099],
               [-1.6821,  1.0561,  1.8823],
               [ 0.2163, -0.1810, -0.8099],
               [-1.6821,  1.0561,  1.8823],
               [ 0.2163, -0.1810, -0.8099],
               [-1.6821,  1.0561,  1.8823]]),
    x: Batch(
           c: tensor([[-1.7257, -0.5643, -0.3352, -1.6255],
                      [-0.1530,  0.7911,  0.3412,  0.1411],
                      [ 0.2110,  0.8384,  0.4537,  1.5757],
                      [-1.7257, -0.5643, -0.3352, -1.6255],
                      [-0.1530,  

In [28]:
%timeit Batch.cat(batches)

141 µs ± 209 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

304 µs ± 2.42 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
